<a href="https://colab.research.google.com/github/zhiyuan-95/portfolio_optimizer/blob/main/portfolio_optimizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from pandas_datareader import data as pdr
import yfinance as yfin
from datetime import datetime, timedelta,date
import time
import pandas as pd
import numpy as np

In [4]:
yfin.pdr_override()

In [5]:
def Past3YearAveReturn(ticker,current_date):
  stock = pd.read_csv('{0}.csv'.format(ticker))
  return_sum = 0
  for y in range(3):
    the_year = str(int(current_date[:4])-y-1)
    stock_last_yr= stock.loc[stock.Date.str.startswith(the_year)]
    div = yfin.Ticker(ticker).dividends
    div_sum = sum(div[div.index.strftime('%Y').str.startswith(the_year)])
    p0 = stock_last_yr['Adj Close'][stock_last_yr.index[0]]
    p1 = stock_last_yr['Adj Close'][stock_last_yr.index[-1]]
    Return = (p1+div_sum-p0)/p0
    return_sum+=Return
#  print(p0,p1)
  return return_sum/3

In [8]:
def Expected_Return(ticker,current_date,period):
  return (0.9*returnOfThisPeriod(ticker,current_date,period)+0.1*Past3YearAveReturn(ticker,current_date))

In [10]:
class portfolio:
  def __init__(self,tickers):
    self._tickers = tickers
    self._div_history = pd.DataFrame()
    self._price_history = {}
    self.get_stocks()
    self._t0 = ''
    self._period = 12
    self._t1 = ''

    self._price= pd.DataFrame()
    self._return = []
    self._div = pd.DataFrame()
  def get_stocks(self):
      file_names = []
      temp = []
      div_date = []
      div_history = {}
      for x in self._tickers:
        Start = '2007-1-1'
        End = '2023-1-1'
        df = pdr.get_data_yahoo(x,start=Start, end=End)
        df.to_csv(x+'.csv')
        self._price_history[x] = pd.read_csv('{0}.csv'.format(x))[['Date','Adj Close']]
        self._price_history[x]['Date']=pd.to_datetime(self._price_history[x]['Date'])
        div_history[x] = yfin.Ticker(x).dividends
        div_date+=list(div_history[x].index)
      div_date = sorted(div_date)
      self._div_history.index= div_date
      for y in self._tickers:
        self._div_history[y] =div_history[y]
  def update(self, date):
    self._t1 = datetime.strptime(date,'%Y-%m-%d')
    self._t0 = self._t1-timedelta(weeks = self._period)
    start = self._t0.strftime('%Y-%m-%d')
    end = self._t1.strftime("%Y-%m-%d")
    last_year = str(int(end[:4])-1)
    for ticker in self._tickers:
      self._price[ticker] = self._price_history[ticker].loc[(self._price_history[ticker]['Date'] >= start) & (self._price_history[ticker]['Date'] <= end)][['Adj Close']]
      self._div[ticker] = self._div_history[ticker][self._div_history[ticker].index.strftime('%Y').str.startswith(last_year)]
      self._div[ticker] = self._div[ticker].fillna(0)
    self._price = self._price.to_numpy()
    self._div = self._div.to_numpy()
  def Return(self):
    # return  = return on stock value from t0-t1 + expected on dividend
    # expected return on dividend of period t of year n = sum of dividend of year n-1 *(t/52)
    p0 = self._price[0]
    p1 = self._price[1]
    return_p = (p1-p0)/p0
    div_rate = sum(self._div)/p0
    E_return_d = div_rate*self._period/52
    return return_p+E_return_d
  def Past3YearAveReturn(self):
    for t in range(3):
      the_year = str(int(self._t1[:4])-y-1)
      stock_last_yr= self._price_history.loc[self._price_history.Date.str.startswith(the_year)]
      div_sum = sum(self._div_history[self._div_history.index.strftime('%Y').str.startswith(the_year)])



In [11]:
  def Past3YearAveReturn(self):
    for t in range(3):
      the_year = str(int(self._t1[:4])-y-1)
      stock_last_yr= self._price_history.loc[self._price_history.Date.str.startswith(the_year)]
      div_sum = sum(self._div_history[self._div_history.index.strftime('%Y').str.startswith(the_year)])

    stock = pd.read_csv('{0}.csv'.format(ticker))
    return_sum = 0
    for y in range(3):
      the_year = str(int(current_date[:4])-y-1)
      stock_last_yr= stock.loc[stock.Date.str.startswith(the_year)]
      div = yfin.Ticker(ticker).dividends
      div_sum = sum(div[div.index.strftime('%Y').str.startswith(the_year)])
      p0 = stock_last_yr['Adj Close'][stock_last_yr.index[0]]
      p1 = stock_last_yr['Adj Close'][stock_last_yr.index[-1]]
      Return = (p1+div_sum-p0)/p0
      return_sum+=Return
  #  print(p0,p1)
    return return_sum/3

In [12]:
tickers = ['qqq','dog','spy']
p = portfolio(tickers)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [13]:
p.update('2021-1-1')

In [15]:
p.Return()

array([ 0.03197768, -0.00911109,  0.02003265])

,spy
1993-03-19 00:00:00-05:00,0.213
1993-06-18 00:00:00-04:00,0.318
1993-09-17 00:00:00-04:00,0.286
1993-12-17 00:00:00-05:00,0.317
1994-03-18 00:00:00-05:00,0.271
...,...
2023-03-20 00:00:00-04:00,NaN
2023-03-22 00:00:00-04:00,NaN
2023-06-16 00:00:00-04:00,1.638
2023-06-20 00:00:00-04:00,NaN


In [114]:
a = pd.DataFrame()

In [83]:
div_date = sorted(list(p._div_history['spy'].index)+list(p._div_history['qqq'].index))

In [115]:
a.index = div_date